In [1]:
import keras 
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Deconv2D, Dense, Activation, UpSampling2D, Cropping2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import cv2 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


In [2]:
img_size = [200, 1000,1]
batch_size = 100
epochs = 20 
learning_rate = 0.001
momentum = 0.9


In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(7,7), strides=(1,1), padding='same', activation='relu', input_shape=(200,1000,1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(7,7), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest', ))
model.add(Deconv2D(256, (5,5), strides=(1,1), padding='same'))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Deconv2D(64, (5,5), strides=(1,1), padding='same'))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Deconv2D(32, (5,5), strides=(1,1), padding='same'))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Deconv2D(16, (5,5), strides=(1,1), padding='same'))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Deconv2D(2, (5,5), strides=(1,1), padding='same'))


model.add(Cropping2D(cropping=((12,12), (12, 12))))

model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 1000, 32)     1600      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 500, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 500, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 500, 64)      100416    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 250, 64)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 250, 64)       256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 250, 128)      204928    
__________

In [4]:

data_gen_args = dict(horizontal_flip = True,
                     vertical_flip = True,
                     rotation_range = 2,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     rescale=1.0/255.0,
                     zoom_range=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)


seed = 1


image_generator = image_datagen.flow_from_directory(
    '/home/iali/welds/data/imgs/',
    class_mode=None,
    batch_size=5,
    
    color_mode='grayscale',
    target_size=(200,1000, ),
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    '/home/iali/welds/data/masks',
    class_mode=None,
    color_mode='grayscale',
    batch_size=5,

    target_size=(200, 1000, ),
    seed=seed)


train_generator = zip(image_generator, mask_generator)


Found 36 images belonging to 1 classes.
Found 36 images belonging to 1 classes.


In [5]:

def dice_loss(target,output ): 
    target= tf.squeeze(target, axis=3)
    loss = keras.losses.binary_crossentropy(target, output[:,:,:,0])
    return loss

def accu(y_true, y_pred): 
    y_true= tf.squeeze(y_true, axis=3)
    return keras.metrics.binary_accuracy(y_true, y_pred[:,:,:,0])
opt = SGD(learning_rate, momentum=momentum)
model.compile(opt, loss=dice_loss, metrics=[ accu])

In [204]:
model.fit_generator(
    train_generator,
    steps_per_epoch=5,
   epochs=10
    )

Epoch 1/10
5/5 [==============================] - 4s 812ms/step - loss: 0.1195 - accu: 0.9609
Epoch 2/10
5/5 [==============================] - 4s 740ms/step - loss: 0.2038 - accu: 0.9224
Epoch 3/10
5/5 [==============================] - 5s 936ms/step - loss: 0.1392 - accu: 0.9535
Epoch 4/10
5/5 [==============================] - 4s 775ms/step - loss: 0.1542 - accu: 0.9470
Epoch 5/10
5/5 [==============================] - 5s 921ms/step - loss: 0.1292 - accu: 0.9534
Epoch 6/10
5/5 [==============================] - 4s 763ms/step - loss: 0.1378 - accu: 0.9543
Epoch 7/10
5/5 [==============================] - 4s 768ms/step - loss: 0.1212 - accu: 0.9575
Epoch 8/10
5/5 [==============================] - 5s 918ms/step - loss: 0.1129 - accu: 0.9584
Epoch 9/10
5/5 [==============================] - 4s 763ms/step - loss: 0.1028 - accu: 0.9638
Epoch 10/10
5/5 [==============================] - 4s 758ms/step - loss: 0.1074 - accu: 0.9594


In [207]:
model.save('weld_seg_UNET.h5')
